# Selenium

## 하나씩 실행하기

In [2]:
import pandas as pd 
from io import StringIO
from selenium import webdriver
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
from selenium import webdriver

# 1. URL 접속
driver = webdriver.Chrome()

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver.get(url)                 # URL로 이동
# driver.maximize_window()        # 열린 크롬창 최대화

In [4]:
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

year = 2023

# 2. SelectBox - 년도 카테고리 선택
WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "sSearchYearFrom"))
    )                                                               # 화면이 나타날때까지 대기
select_box = driver.find_element(By.ID, "sSearchYearFrom")          # 년도 카테고리 접근
select_obj = Select(select_box)                                     # 선택할 수 있는 상태로 변경
select_obj.select_by_value(str(year))                               # 내가 원하는 년도 선택

In [10]:
# 3. 조회 버튼 클릭
search_btn = driver.find_element(By.CLASS_NAME, "wrap_btn") # 조회 버튼 접근
search_btn.click()                                          # 조회 버튼 클릭

In [11]:
import pandas as pd 
from io import StringIO

# 4. 박스오피스 통계 데이터 추출
## 1) HTML 추출
html_text = driver.page_source
print(html_text)

<html lang="ko"><head>
<meta charset="utf-8">
<title>KOFIC 영화관 입장권 통합전산망 :: 연도별 박스오피스</title>
<!-- 공통 js/css -->













<!--[if IE]>
<script src="/kobis/web/comm/commjs/html5shiv.js"></script>
<![endif]-->



<!-- 기본 css -->
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/default.css">
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/jquery-ui-1.12.1.min.css">
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/common.css">


<script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=UA-127072686-1&amp;l=dataLayer&amp;cx=c&amp;gtm=45je4cc1v9118810064za200"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-62TRNDZ0CK&amp;cx=c&amp;_slc=1"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-62TRNDZ0CK

In [12]:
## 2) pd.read_html을 통해 데이터프레임 변환
html_io = StringIO(html_text)
tables = pd.read_html(html_io)
print(f"데이터프레임 개수: {len(tables)}")

데이터프레임 개수: 1


In [14]:
## 3) 데이터프레임 만들기
movie_data1 = tables[0]
movie_data1 = movie_data1.set_index("순위")
movie_data1 = movie_data1.rename(
    columns={
        "매출액  오름차순  내림차순": "매출액",
        "매출액 점유율  오름차순  내림차순": "매출액 점유율",
        "관객수  오름차순  내림차순": "관객수",
        "스크린수  오름차순  내림차순": "스크린수",
        "상영횟수  오름차순  내림차순": "상영횟수",
    }
)
movie_data1.head(10)

,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,상영횟수
순위,,,,,,,
1,서울의 봄,2023-11-22,115350491410,9.1%,11854779,2463,292299
2,범죄도시3,2023-05-31,104687737771,8.3%,10682813,2582,311915
3,엘리멘탈,2023-06-14,71120500040,5.6%,7238453,1664,247169
4,스즈메의 문단속,2023-03-08,57106531498,4.5%,5574358,1572,244575
5,밀수,2023-07-26,49685009003,3.9%,5143219,1932,167669
6,더 퍼스트 슬램덩크,2023-01-04,50057140100,4.0%,4786406,1060,231062
7,가디언즈 오브 갤럭시: Volume 3,2023-05-03,44357156661,3.5%,4209118,1819,153711
8,미션 임파서블: 데드 레코닝 PART ONE,2023-07-12,40428788113,3.2%,4022072,2437,137933
9,콘크리트 유토피아,2023-08-09,37412802232,3.0%,3849242,1731,150918


In [15]:
## 4) 데이터프레임 저장
movie_data1.to_csv("boxoffice_top50_stats.csv", index=False)

In [28]:
# 5. 영화정보 데이터 추출(한개만)
## 1) 영화 링크 클릭
link_css = "#td_movie > span > a"
links = driver.find_elements(By.CSS_SELECTOR, link_css)
link = links[0]
link.click()          # 영화 제목 클릭

title = link.text     # 영화 제목 저장
print(title)

서울의 봄


In [17]:
## 2) 새로운 창으로 열리는 건지 확인
print(driver.current_window_handle)     # 현재 윈도우 창 출력
print(driver.window_handles)            # 켜져있는 윈도우 창 리스트 출력

73B2415E3A92A849258CB1CBE34E477D
['73B2415E3A92A849258CB1CBE34E477D']


In [283]:
### 만약에 창이 2개 이상이라면?
# handles = driver.window_handles
# driver.switch_to.window(handles[-1])    # 최근에 켜진 창으로 페이지 소스 이동

In [19]:
## 3) 정보 추출

### 3-1) 영화 포스터 추출
poster_css = "a.fl.thumb"
WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, poster_css))
    )                                                                               # 화면이 나타날때까지 대기
poster = driver.find_element(By.CSS_SELECTOR, poster_css).get_attribute("href")
poster

'https://www.kobis.or.kr/common/mast/movie/2023/11/2d4842b5ae044adfaa2769ab51542ab8.jpg'

In [20]:
### 3-2) 영화 코드 추출
code_css = "div.item_tab.basic > div.ovf.info.info1 > dl > dd:nth-child(2)"
code = driver.find_element(By.CSS_SELECTOR, code_css).text
code

'20212866'

In [22]:
### 3-3) 영화 상영정보 추출
summary_css = "div.item_tab.basic > div.ovf.info.info1 > dl > dd:nth-child(8)"
summary = driver.find_element(By.CSS_SELECTOR, summary_css).text
summary

'장편 | 일반영화 | 드라마 | 141분 15초 | 12세이상관람가 | 한국 | 한글자막(CC) , 화면해설(AD)'

In [23]:
#### 분리
summary_list = summary.split(" | ")
if len(summary_list) == 6:
    _, _, genre, running_time, age_limit, nation = summary_list
else:
    _, _, genre, running_time, age_limit, nation, _ = summary_list
    
print(genre, running_time, age_limit, nation)

드라마 141분 15초 12세이상관람가 한국


In [24]:
### 3-4) 영화 출연진 추출
director_css = "div.staffMore > dl > div:nth-child(1) > dd > a"
staff_css = "div.staffMore > dl > div:nth-child(2) > dd > table:nth-child(1) > tbody > tr > td > a"

director = ", ".join([x.text for x in driver.find_elements(By.CSS_SELECTOR, director_css)])
staff = ", ".join([x.text for x in driver.find_elements(By.CSS_SELECTOR, staff_css)])

print(director)
print(staff)

김성수
황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태건), 김성균(헌병감 김준엽)


In [25]:
## 4) 모든 데이터 수집
info_dict = {
    "영화명": title,
    "장르": genre,
    "상영시간": running_time,
    "연령제한": age_limit,
    "국적": nation,
    "감독": director,
    "주연배우": staff
}
info_dict

{'영화명': '서울의 봄',
 '장르': '드라마',
 '상영시간': '141분 15초',
 '연령제한': '12세이상관람가',
 '국적': '한국',
 '감독': '김성수',
 '주연배우': '황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태건), 김성균(헌병감 김준엽)'}

In [29]:
## 5) 뒤로가기 클릭
back_css = "body > div.ui-dialog.ui-corner-all.ui-widget.ui-widget-content.ui-front.ui-draggable.ui-resizable > div.ui-dialog-titlebar.ui-corner-all.ui-widget-header.ui-helper-clearfix.ui-draggable-handle > div.hd_layer > a.close.back > span"
back_btn = driver.find_element(By.CSS_SELECTOR, back_css)
back_btn.click()

## 데이터 수집 반복문

In [30]:
# 5. 영화정보 데이터 추출(모든 리스트)
## css selector 모음
link_css = "#td_movie > span > a"
poster_css = "a.fl.thumb"
code_css = "div.item_tab.basic > div.ovf.info.info1 > dl > dd:nth-child(2)"
summary_css = "div.item_tab.basic > div.ovf.info.info1 > dl > dd:nth-child(8)"
director_css = "div.staffMore > dl > div:nth-child(1) > dd > a"
staff_css = "div.staffMore > dl > div:nth-child(2) > dd > table:nth-child(1) > tbody > tr > td > a"
back_css = "body > div.ui-dialog.ui-corner-all.ui-widget.ui-widget-content.ui-front.ui-draggable.ui-resizable > div.ui-dialog-titlebar.ui-corner-all.ui-widget-header.ui-helper-clearfix.ui-draggable-handle > div.hd_layer > a.close.back > span"

## 1) 영화 링크 리스트 추출
movie_list = []
links = driver.find_elements(By.CSS_SELECTOR, link_css)

for rank, link in enumerate(links):
    ## 2) 영화 링크 클릭
    title = link.text     # 영화 제목 저장
    link.click()          # 영화 클릭
    
    ## 3) 정보 추출
    ### 3-1) 영화 포스터 추출
    WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, code_css))
        )                                                                               # 화면이 나타날때까지 대기
    poster = driver.find_element(By.CSS_SELECTOR, poster_css).get_attribute("href")
    
    ### 3-2) 영화 코드 추출
    code = driver.find_element(By.CSS_SELECTOR, code_css).text
    
    ### 3-3) 영화 상영정보 추출
    summary = driver.find_element(By.CSS_SELECTOR, summary_css).text
    summary_list = summary.split(" | ")
    if len(summary_list) == 6:
        _, _, genre, running_time, age_limit, nation = summary.split(" | ")
    else:
        _, _, genre, running_time, age_limit, nation, _ = summary.split(" | ")

    ### 3-4) 영화 출연진 추출
    director = ", ".join([x.text for x in driver.find_elements(By.CSS_SELECTOR, director_css)])
    staff = ", ".join([x.text for x in driver.find_elements(By.CSS_SELECTOR, staff_css)])

    ## 4) 데이터 수집
    info_dict = {
        "연도": year,
        "순위": rank+1,
        "영화코드": code,
        "영화명": title, 
        "장르": genre,
        "상영시간": running_time,
        "연령제한": age_limit,
        "국적": nation,
        "감독": director,
        "주연배우": staff 
    }
    movie_list.append(info_dict)
    print(info_dict)

    ## 5) 뒤로가기 클릭
    back_btn = driver.find_element(By.CSS_SELECTOR, back_css)
    back_btn.click()
        

{'연도': 2023, '순위': 1, '영화코드': '20212866', '영화명': '서울의 봄', '장르': '드라마', '상영시간': '141분 15초', '연령제한': '12세이상관람가', '국적': '한국', '감독': '김성수', '주연배우': '황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태건), 김성균(헌병감 김준엽)'}
{'연도': 2023, '순위': 2, '영화코드': '20226411', '영화명': '범죄도시3', '장르': '범죄, 액션', '상영시간': '105분 7초', '연령제한': '15세이상관람가', '국적': '한국', '감독': '이상용', '주연배우': '마동석(마석도), 이준혁(주성철), 아오키 무네타카(리키), 마동석, 김병탁'}
{'연도': 2023, '순위': 3, '영화코드': '20228930', '영화명': '엘리멘탈', '장르': '애니메이션', '상영시간': '109분 9초', '연령제한': '전체관람가', '국적': '미국', '감독': '피터 손', '주연배우': ''}
{'연도': 2023, '순위': 4, '영화코드': '20226270', '영화명': '스즈메의 문단속', '장르': '애니메이션', '상영시간': '121분 47초', '연령제한': '12세이상관람가', '국적': '일본', '감독': '신카이 마코토', '주연배우': '하라 나노카, 마츠무라 호쿠토'}
{'연도': 2023, '순위': 5, '영화코드': '20218541', '영화명': '밀수', '장르': '액션', '상영시간': '129분 3초', '연령제한': '15세이상관람가', '국적': '한국', '감독': '류승완', '주연배우': '김혜수(조춘자), 염정아(엄진숙), 조인성(권 상사), 박정민(장도리), 김종수(이장춘), 고민시(고옥분)'}
{'연도': 2023, '순위': 6, '영화코드': '20228555', '영화명': '더 퍼스트 슬램덩크', '장르': '애니메이션',

In [246]:
## 7) 데이터프레임 만들기
movie_data2 = pd.DataFrame(movie_list)

## 8) 데이터프레임 저장
movie_data2.to_csv("boxoffice_top50_info.csv", index=False)

## 최종 반복문

In [ ]:
import pandas as pd 
from io import StringIO
from selenium import webdriver
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#---------------------------------------------------------------------------------------------
# 변수 설정
#---------------------------------------------------------------------------------------------
year = 2023
file_path1 = "boxoffice_top50_stats.csv"
file_path2 = "boxoffice_top50_info.csv"

#---------------------------------------------------------------------------------------------
# 1. URL 접속
#---------------------------------------------------------------------------------------------
driver = webdriver.Chrome()

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver.get(url)                 # URL로 이동
driver.maximize_window()        # 열린 크롬창 최대화

from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#---------------------------------------------------------------------------------------------
# 2. SelectBox - 년도 카테고리 선택
#---------------------------------------------------------------------------------------------
WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.ID, "sSearchYearFrom"))
    )                                                               # 화면이 나타날때까지 대기
select_box = driver.find_element(By.ID, "sSearchYearFrom")          # 년도 카테고리 접근
select_obj = Select(select_box)                                     # 선택할 수 있는 상태로 변경
select_obj.select_by_value(str(year))                               # 내가 원하는 년도 선택

#---------------------------------------------------------------------------------------------
# 3. 조회 버튼 클릭
#---------------------------------------------------------------------------------------------
search_btn = driver.find_element(By.CLASS_NAME, "wrap_btn")         # 조회 버튼 접근
search_btn.click()                                                  # 조회 버튼 클릭

#---------------------------------------------------------------------------------------------
# 4. 박스오피스 통계 데이터 추출
#---------------------------------------------------------------------------------------------
## 1) HTML 추출
html_text = driver.page_source

## 2) pd.read_html을 통해 데이터프레임 변환
html_io = StringIO(html_text)
tables = pd.read_html(html_io)

## 3) 데이터프레임 만들기
movie_data1 = pd.DataFrame(tables[0])
movie_data1 = movie_data1.set_index("순위")
movie_data1 = movie_data1.rename(
    columns={
        "매출액  오름차순  내림차순": "매출액",
        "매출액 점유율  오름차순  내림차순": "매출액 점유율",
        "관객수  오름차순  내림차순": "관객수",
        "스크린수  오름차순  내림차순": "스크린수",
        "상영횟수  오름차순  내림차순": "상영횟수",
    }
)
movie_data1.head(10)

## 4) 데이터프레임 저장
movie_data1.to_csv(file_path1, index=False)
print(f"박스오피스 Top50 통계 데이터가 저장되었습니다(filepath: {file_path1})")

#---------------------------------------------------------------------------------------------
# 5. 영화정보 데이터 추출
#---------------------------------------------------------------------------------------------
## css selector 모음
link_css = "#td_movie > span > a"
poster_css = "a.fl.thumb"
code_css = "div.item_tab.basic > div.ovf.info.info1 > dl > dd:nth-child(2)"
summary_css = "div.item_tab.basic > div.ovf.info.info1 > dl > dd:nth-child(8)"
director_css = "div.staffMore > dl > div:nth-child(1) > dd > a"
staff_css = "div.staffMore > dl > div:nth-child(2) > dd > table:nth-child(1) > tbody > tr > td > a"
back_css = "body > div.ui-dialog.ui-corner-all.ui-widget.ui-widget-content.ui-front.ui-draggable.ui-resizable > div.ui-dialog-titlebar.ui-corner-all.ui-widget-header.ui-helper-clearfix.ui-draggable-handle > div.hd_layer > a.close.back > span"

## 1) 영화 링크 리스트 추출
movie_list = []
links = driver.find_elements(By.CSS_SELECTOR, link_css)

for rank, link in enumerate(links):
    ## 2) 영화 링크 클릭
    title = link.text     # 영화 제목 저장
    link.click()          # 영화 클릭
    
    ## 3) 정보 추출
    ### 3-1) 영화 포스터 추출
    WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, code_css))
        )                                                                               # 화면이 나타날때까지 대기
    poster = driver.find_element(By.CSS_SELECTOR, poster_css).get_attribute("href")
    
    ### 3-2) 영화 코드 추출
    code = driver.find_element(By.CSS_SELECTOR, code_css).text
    
    ### 3-3) 영화 상영정보 추출
    summary = driver.find_element(By.CSS_SELECTOR, summary_css).text
    summary_list = summary.split(" | ")
    if len(summary_list) == 6:
        _, _, genre, running_time, age_limit, nation = summary.split(" | ")
    else:
        _, _, genre, running_time, age_limit, nation, _ = summary.split(" | ")

    ### 3-4) 영화 출연진 추출
    director = ", ".join([x.text for x in driver.find_elements(By.CSS_SELECTOR, director_css)])
    staff = ", ".join([x.text for x in driver.find_elements(By.CSS_SELECTOR, staff_css)])

    ## 4) 데이터 수집
    info_dict = {
        "연도": year,
        "순위": rank+1,
        "영화코드": code,
        "영화명": title, 
        "장르": genre,
        "상영시간": running_time,
        "연령제한": age_limit,
        "국적": nation,
        "감독": director,
        "주연배우": staff 
    }
    movie_list.append(info_dict)
    print(info_dict)

    ## 5) 뒤로가기 클릭
    back_btn = driver.find_element(By.CSS_SELECTOR, back_css)
    back_btn.click()

## 7) 데이터프레임 만들기
movie_data2 = pd.DataFrame(movie_list)
movie_data2 = movie_data2.set_index("순위")

## 8) 데이터프레임 저장
movie_data2.to_csv(file_path2, index=False)
print(f"박스오피스 Top50 영화 정보 데이터가 저장되었습니다(filepath: {file_path2})")

#---------------------------------------------------------------------------------------------
# 윈도우 창 닫기
#---------------------------------------------------------------------------------------------
driver.close()        

박스오피스 Top50 통계 데이터가 저장되었습니다(filepath: boxoffice_top50_stats.csv)
{'연도': 2023, '순위': 1, '영화코드': '20212866', '영화명': '서울의 봄', '장르': '드라마', '상영시간': '141분 15초', '연령제한': '12세이상관람가', '국적': '한국', '감독': '김성수', '주연배우': '황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태건), 김성균(헌병감 김준엽)'}
{'연도': 2023, '순위': 2, '영화코드': '20226411', '영화명': '범죄도시3', '장르': '범죄, 액션', '상영시간': '105분 7초', '연령제한': '15세이상관람가', '국적': '한국', '감독': '이상용', '주연배우': '마동석(마석도), 이준혁(주성철), 아오키 무네타카(리키), 마동석, 김병탁'}
{'연도': 2023, '순위': 3, '영화코드': '20228930', '영화명': '엘리멘탈', '장르': '애니메이션', '상영시간': '109분 9초', '연령제한': '전체관람가', '국적': '미국', '감독': '피터 손', '주연배우': ''}
{'연도': 2023, '순위': 4, '영화코드': '20226270', '영화명': '스즈메의 문단속', '장르': '애니메이션', '상영시간': '121분 47초', '연령제한': '12세이상관람가', '국적': '일본', '감독': '신카이 마코토', '주연배우': '하라 나노카, 마츠무라 호쿠토'}
{'연도': 2023, '순위': 5, '영화코드': '20218541', '영화명': '밀수', '장르': '액션', '상영시간': '129분 3초', '연령제한': '15세이상관람가', '국적': '한국', '감독': '류승완', '주연배우': '김혜수(조춘자), 염정아(엄진숙), 조인성(권 상사), 박정민(장도리), 김종수(이장춘), 고민시(고옥분)'}
{'연도': 2023,

## 데이터 분석

### 데이터 확인

In [261]:
movie_data1.head()

,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,상영횟수
순위,,,,,,,
1,서울의 봄,2023-11-22,115350491410,9.1%,11854779,2463,292299
2,범죄도시3,2023-05-31,104687737771,8.3%,10682813,2582,311915
3,엘리멘탈,2023-06-14,71120500040,5.6%,7238453,1664,247169
4,스즈메의 문단속,2023-03-08,57106531498,4.5%,5574358,1572,244575
5,밀수,2023-07-26,49685009003,3.9%,5143219,1932,167669


In [262]:
movie_data2.head()

,연도,영화코드,영화명,장르,상영시간,연령제한,국적,감독,주연배우
순위,,,,,,,,,
1,2023,20212866,서울의 봄,드라마,141분 15초,12세이상관람가,한국,김성수,"황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태..."
2,2023,20226411,범죄도시3,"범죄, 액션",105분 7초,15세이상관람가,한국,이상용,"마동석(마석도), 이준혁(주성철), 아오키 무네타카(리키), 마동석, 김병탁"
3,2023,20228930,엘리멘탈,애니메이션,109분 9초,전체관람가,미국,피터 손,
4,2023,20226270,스즈메의 문단속,애니메이션,121분 47초,12세이상관람가,일본,신카이 마코토,"하라 나노카, 마츠무라 호쿠토"
5,2023,20218541,밀수,액션,129분 3초,15세이상관람가,한국,류승완,"김혜수(조춘자), 염정아(엄진숙), 조인성(권 상사), 박정민(장도리), 김종수(이..."


### 데이터 분석 I - 상영시간이 가장 긴 순으로 내림차순 

In [263]:
## 상영시간 초로 바꾸기 
def calc_seconds(running_time):
    minutes, seconds = running_time.split()

    return int(minutes[:-1]) * 60 + int(seconds[:-1])

movie_data2["상영시간2"] = movie_data2["상영시간"].apply(lambda x: calc_seconds(x))
movie_data2.head()

,연도,영화코드,영화명,장르,상영시간,연령제한,국적,감독,주연배우,상영시간2
순위,,,,,,,,,,
1,2023,20212866,서울의 봄,드라마,141분 15초,12세이상관람가,한국,김성수,"황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태...",8475
2,2023,20226411,범죄도시3,"범죄, 액션",105분 7초,15세이상관람가,한국,이상용,"마동석(마석도), 이준혁(주성철), 아오키 무네타카(리키), 마동석, 김병탁",6307
3,2023,20228930,엘리멘탈,애니메이션,109분 9초,전체관람가,미국,피터 손,,6549
4,2023,20226270,스즈메의 문단속,애니메이션,121분 47초,12세이상관람가,일본,신카이 마코토,"하라 나노카, 마츠무라 호쿠토",7307
5,2023,20218541,밀수,액션,129분 3초,15세이상관람가,한국,류승완,"김혜수(조춘자), 염정아(엄진숙), 조인성(권 상사), 박정민(장도리), 김종수(이...",7743


In [264]:
## 상영시간이 가장 긴 영화 
movie_data2.sort_values(by="상영시간2", ascending=False).head()

,연도,영화코드,영화명,장르,상영시간,연령제한,국적,감독,주연배우,상영시간2
순위,,,,,,,,,,
49,2023,19980074,타이타닉,"드라마, 멜로/로맨스, 액션",194분 51초,15세이상관람가,미국,제임스 카메론,"레오나르도 디카프리오, 케이트 윈슬렛",11691
10,2023,20225061,아바타: 물의 길,"액션, 어드벤처, SF, 스릴러",192분 24초,12세이상관람가,미국,제임스 카메론,"조 샐다나, 샘 워싱턴, 시고니 위버, 스티븐 랭, 케이트 윈슬렛",11544
12,2023,20226431,오펜하이머,"스릴러, 드라마",180분 9초,15세이상관람가,미국,크리스토퍼 놀란,"킬리언 머피, 에밀리 블런트, 맷 데이먼, 로버트 다우니 주니어, 플로렌스 퓨, 조...",10809
16,2023,20231089,존 윅 4,액션,169분 5초,청소년관람불가,미국,채드 스타헬스키,"키아누 리브스, 로렌스 피쉬번, 이안 맥쉐인, 빌 스카스가드, 견자단",10145
8,2023,20232631,미션 임파서블: 데드 레코닝 PART ONE,"액션, 스릴러, 어드벤처",163분 13초,15세이상관람가,미국,크리스토퍼 맥쿼리,"톰 크루즈, 헤일리 앳웰, 빙 라메스, 사이먼 페그, 레베카 퍼거슨, 바네사 커비,...",9793


### 데이터 분석 II - 인기많은 장르 파악

In [265]:
## 올해의 장르
genre_list = []
for genre in movie_data2["장르"]:
    genre_list.extend(genre.split(", "))

from collections import Counter
genre_counter = Counter(genre_list)
print(genre_counter)

Counter({'액션': 19, '드라마': 15, '애니메이션': 11, '스릴러': 5, '어드벤처': 5, '코미디': 4, '판타지': 4, '범죄': 3, 'SF': 3, '멜로/로맨스': 3, '기타': 2, '뮤지컬': 2, '사극': 1, '미스터리': 1, '공포(호러)': 1, '가족': 1})


In [266]:
## 조회
movie_data2.loc[movie_data2["장르"].str.contains("애니메이션")]

,연도,영화코드,영화명,장르,상영시간,연령제한,국적,감독,주연배우,상영시간2
순위,,,,,,,,,,
3,2023,20228930,엘리멘탈,애니메이션,109분 9초,전체관람가,미국,피터 손,,6549
4,2023,20226270,스즈메의 문단속,애니메이션,121분 47초,12세이상관람가,일본,신카이 마코토,"하라 나노카, 마츠무라 호쿠토",7307
6,2023,20228555,더 퍼스트 슬램덩크,애니메이션,124분 26초,12세이상관람가,일본,이노우에 다케히코,,7466
13,2023,20227890,슈퍼 마리오 브라더스,"애니메이션, 어드벤처, 코미디",92분 17초,전체관람가,미국,,"크리스 프랫(슈퍼 마리오), 안야 테일러 조이(피치 공주), 잭 블랙(쿠파), 찰리...",5537
15,2023,20234664,그대들은 어떻게 살 것인가,애니메이션,123분 47초,전체관람가,일본,미야자키 하야오,"스다 마사키, 시바사키 코우, 키무라 요시노, 기무라 타쿠야, 산토키 소마, 아이묭",7427
27,2023,20232180,스파이더맨: 어크로스 더 유니버스,애니메이션,139분 55초,전체관람가,미국,호아킴 도스 산토스,"샤메익 무어, 헤일리 스테인펠드, 제이크 존슨, 브라이언 타이리 헨리, 오스카 아이삭",8395
29,2023,20224109,장화신은 고양이: 끝내주는 모험,애니메이션,102분 14초,전체관람가,미국,조엘 크로포드,"안토니오 반데라스, 셀마 헤이엑, 올리비아 콜맨, 하비 길렌, 샘슨 카요, 와그너 ...",6134
30,2023,20232536,명탐정코난: 흑철의 어영,애니메이션,109분 35초,12세이상관람가,일본,타치카와 유즈루,,6575
33,2023,20231348,극장판 짱구는 못말려: 동물소환 닌자 배꼽수비대,애니메이션,99분 45초,전체관람가,일본,하시모토 마사카즈,,5985


### 데이터 분석 III - 가장 많이 출연한 배우

In [267]:
## 올해의 배우
actor_list = []
for actor in movie_data2["주연배우"]:
    actors = actor.split(", ")
    for act in actors:
        if act == "":
            continue
        actor_list.append(act.split("(")[0].strip())

from collections import Counter
actor_counter = Counter(actor_list)
print(actor_counter)

Counter({'박서준': 3, '설경구': 3, '황정민': 2, '이성민': 2, '김성균': 2, '마동석': 2, '김종수': 2, '크리스 프랫': 2, '조 샐다나': 2, '폼 클레멘티에프': 2, '빈 디젤': 2, '케이트 윈슬렛': 2, '허준호': 2, '플로렌스 퓨': 2, '제이슨 모모아': 2, '브리 라슨': 2, '강기영': 2, '하정우': 2, '정우성': 1, '박해준': 1, '이준혁': 1, '아오키 무네타카': 1, '김병탁': 1, '하라 나노카': 1, '마츠무라 호쿠토': 1, '김혜수': 1, '염정아': 1, '조인성': 1, '박정민': 1, '고민시': 1, '데이브 바티스타': 1, '카렌 길런': 1, '브래들리 쿠퍼': 1, '윌 폴터': 1, '톰 크루즈': 1, '헤일리 앳웰': 1, '빙 라메스': 1, '사이먼 페그': 1, '레베카 퍼거슨': 1, '바네사 커비': 1, '에사이 모랄레스': 1, '이병헌': 1, '박보영': 1, '샘 워싱턴': 1, '시고니 위버': 1, '스티븐 랭': 1, '김윤석': 1, '백윤식': 1, '정재영': 1, '킬리언 머피': 1, '에밀리 블런트': 1, '맷 데이먼': 1, '로버트 다우니 주니어': 1, '조쉬 하트넷': 1, '케이시 애플렉': 1, '라미 말렉': 1, '케네스 브래너': 1, '안야 테일러 조이': 1, '잭 블랙': 1, '찰리 데이': 1, '강하늘': 1, '정소민': 1, '조민수': 1, '김선영': 1, '스다 마사키': 1, '시바사키 코우': 1, '키무라 요시노': 1, '기무라 타쿠야': 1, '산토키 소마': 1, '아이묭': 1, '키아누 리브스': 1, '로렌스 피쉬번': 1, '이안 맥쉐인': 1, '빌 스카스가드': 1, '견자단': 1, '강동원': 1, '이솜': 1, '이동휘': 1, '정성화': 1, '김고은': 1, '나문희': 1, '조재윤': 1, '배정남': 1, '이현우': 1, '박

In [268]:
## 조회
movie_data2.loc[movie_data2["주연배우"].str.contains("박서준|설경구")]

,연도,영화코드,영화명,장르,상영시간,연령제한,국적,감독,주연배우,상영시간2
순위,,,,,,,,,,
9,2023,20204264,콘크리트 유토피아,"드라마, 기타",129분 59초,15세이상관람가,한국,엄태화,"이병헌, 박서준, 박보영",7799
24,2023,20198482,드림,"코미디, 드라마",125분 1초,12세이상관람가,한국,이병헌,"박서준(윤홍대), 아이유(이소민)",7501
37,2023,20235034,더 마블스,"액션, SF",104분 36초,12세이상관람가,미국,니아 다코스타,"브리 라슨, 테요나 패리스, 이만 벨라니, 자웨 애쉬튼, 사무엘 L. 잭슨, 박서준",6276
39,2023,20214823,유령,"드라마, 액션",132분 34초,15세이상관람가,한국,이해영,"설경구, 이하늬, 박소담, 박해수, 서현우",7954
45,2023,20218935,더 문,"SF, 액션, 드라마",129분 20초,12세이상관람가,한국,김용화,"설경구(김재국), 도경수(황선우), 박병은(센터장), 조한철(장관), 최병모(차관)...",7760
48,2023,20200622,소년들,"드라마, 범죄",123분 59초,15세이상관람가,한국,정지영,"설경구, 유준상, 진경, 허성태, 염혜란",7439


In [269]:
total_data = pd.merge(movie_data1, movie_data2, how="left")
total_data.head()

,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,상영횟수,연도,영화코드,장르,상영시간,연령제한,국적,감독,주연배우,상영시간2
0,서울의 봄,2023-11-22,115350491410,9.1%,11854779,2463,292299,2023,20212866,드라마,141분 15초,12세이상관람가,한국,김성수,"황정민(전두광), 정우성(이태신), 이성민(참모총장 정상호), 박해준(9사단장 노태...",8475
1,범죄도시3,2023-05-31,104687737771,8.3%,10682813,2582,311915,2023,20226411,"범죄, 액션",105분 7초,15세이상관람가,한국,이상용,"마동석(마석도), 이준혁(주성철), 아오키 무네타카(리키), 마동석, 김병탁",6307
2,엘리멘탈,2023-06-14,71120500040,5.6%,7238453,1664,247169,2023,20228930,애니메이션,109분 9초,전체관람가,미국,피터 손,,6549
3,스즈메의 문단속,2023-03-08,57106531498,4.5%,5574358,1572,244575,2023,20226270,애니메이션,121분 47초,12세이상관람가,일본,신카이 마코토,"하라 나노카, 마츠무라 호쿠토",7307
4,밀수,2023-07-26,49685009003,3.9%,5143219,1932,167669,2023,20218541,액션,129분 3초,15세이상관람가,한국,류승완,"김혜수(조춘자), 염정아(엄진숙), 조인성(권 상사), 박정민(장도리), 김종수(이...",7743


In [270]:
total_data.to_csv("boxoffice_top50.csv", index=False)

# 자주 쓰는 기능들

## 데이터 입력하기

In [43]:
from selenium import webdriver

# 1. URL 접속
driver = webdriver.Chrome()

url = "https://www.naver.com"
driver.get(url)                 # URL로 이동
# driver.maximize_window()        # 열린 크롬창 최대화

In [55]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# input_box = driver.find_element(By.ID, "query")
input_box = driver.find_element(By.CSS_SELECTOR, "#query")

# 입력하기
input_box.send_keys("인공지능")

In [49]:
# 다시쓰기
input_box.send_keys(Keys.CONTROL + "A")     # 입력 텍스트 전체 블록
input_box.send_keys("인공 지능") 

In [56]:
# 검색 버튼 클릭
search_btn = driver.find_element(By.CSS_SELECTOR, "#search-btn")  # 검색 버튼 접근
search_btn.click()

In [62]:
# 스크롤 다운
body = driver.find_element(By.TAG_NAME, "body")
body.send_keys(Keys.PAGE_DOWN)

In [54]:
# 뒤로가기
driver.back()

## 스크롤 다운

In [63]:
from selenium import webdriver
import urllib

# 1. URL 접속
driver = webdriver.Chrome()

keyword = "세탁기"
keyword_enc = urllib.parse.quote(keyword)
url = f"https://search.shopping.naver.com/ns/search?query={keyword_enc}"
driver.get(url)                 # URL로 이동
driver.maximize_window()        # 열린 크롬창 최대화

In [127]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

body = driver.find_element(By.TAG_NAME, "body")

# PAGE DOWN 키보드 키 누르기
body.send_keys(Keys.PAGE_DOWN)

In [36]:
from selenium import webdriver
import urllib

# 1. URL 접속
driver = webdriver.Chrome()

keyword = "세탁기"
keyword_enc = urllib.parse.quote(keyword)
url = f"https://search.shopping.naver.com/ns/search?query={keyword_enc}"
driver.get(url)                 # URL로 이동
# driver.maximize_window()        # 열린 크롬창 최대화

In [226]:
# 스크롤 끝까지 내리기
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [38]:
from selenium import webdriver
import urllib

# 1. URL 접속
driver = webdriver.Chrome()

keyword = "세탁기"
keyword_enc = urllib.parse.quote(keyword)
url = f"https://search.shopping.naver.com/ns/search?query={keyword_enc}"
driver.get(url)                 # URL로 이동
driver.maximize_window()        # 열린 크롬창 최대화

In [228]:
import time 

# 스크롤 반복해서 내리기
## 최대 반복 수 설정
max_cnt = 500

## 이전 높이 계산
old_height = driver.execute_script("return document.body.scrollHeight")

cnt = 0
while True:
    ## 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    ## 대기
    print(cnt)
    time.sleep(1)
    
    ## 새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")
    ## 더 이상 스크롤되지 않으면 종료
    if new_height == old_height:
        break  
    ## 새로운 높이 계산 값을 이전 높이 변수에 저장장 
    old_height = new_height

    ## 카운트 계산
    if cnt == max_cnt:
        break 
    cnt += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
